<font color="#CA3532"><h1 align="left">**Aprendizaje por Refuerzo**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Multiarmed Bandits Contextuales**</h2></font>

## **Instrucciones**

Ajusta este notebook (igual que el 1) al nuevo problema, donde tenemos 3 productos y 4 variables de contexto.

In [ ]:
COLAB = True

In [ ]:
!ls

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1mSlAdoi6cv2QaUy676LZSpRYdd8ncJHc',
                                        dest_path='./simula_clientes_problema1.py')
    gdd.download_file_from_google_drive(file_id='1ZEYZXdC7GyHXqnt5Wl_orkHmc97vWfGO',
                                        dest_path='./spacebandits.zip', unzip=True)

In [ ]:
import numpy as np
import pandas as pd
from random import random, randint
import matplotlib.pyplot as plt
from tqdm import tqdm

%config InlineBackend.figure_format='retina'
%matplotlib inline

In [ ]:
from simula_clientes_problema1 import cliente

In [ ]:
c = cliente() # x es una instancia de la clase "cliente", que tengo implementada en simula_clientes_problema1

In [ ]:
# edad, ingresos generados en la compañía hasta ahora
c.get_context()

In [ ]:
accion = 2
c.get_reward(accion)

### **Estrategia 1: recomendación al azar**

In [ ]:
N = 10000
reward_total = 0
for _ in range(N):
  c = cliente()
  accion = np.random.choice(3)
  reward_total += c.get_reward(accion)

print("Reward promedio:", reward_total/N)

### **Estrategia 2: Multiarmed Bandit contextual**

In [ ]:
from space_bandits import LinearBandits

n_acciones = 3 # número de promociones diferentes
n_features = 2 # número de variables de contexto (edad e ingresos generados)

N = 10000

agente = LinearBandits(n_acciones, n_features, initial_pulls=100) # initial_pulls: duración de la fase de exploración pura
reward_promedio = []
reward_total = 0
for i in tqdm(range(N)):
  c = cliente() # cliente con el que contacto
  contexto = c.get_context() # consigo el contexto del cliente
  accion = agente.action(contexto) # choose_action
  reward = c.get_reward(accion) # reward obtenido
  agente.update(contexto, accion, reward) # actualización del modelo

  reward_total += reward
  reward_promedio.append(reward_total/(i+1)) # i+1 es el número de iteraciones

print("Reward promedio:", reward_total/N)
plt.plot(reward_promedio)
plt.title("Evolución del reward promedio");

In [ ]:
c = cliente()

In [ ]:
contexto = c.get_context()
contexto

In [ ]:
agente.action(contexto) # Le pido al multiarmed bandit contextual que elija la acción óptima:

In [ ]:
agente.expected_values(contexto) # Q(s,a) por cada acción a, y para el cliente s (s=contexto)

In [ ]:
# Parámetros que aprende el agente:
# 3 arrays (porque hay 3 acciones posibles, 3 productos)
# En cada array el último elemento es el término constante de un modelo lineal
# Los elementos primero y segundo son los coeficientes de edad e ingresos respectivamente
agente.mu

In [ ]:
edad, ingresos_generados = contexto
edad, ingresos_generados

In [ ]:
agente.mu[0]

In [ ]:
# Valoración Q(s,a), según el modelo lineal, de realizar la
# acción 0 (promoción 0) en ese cliente concreto:
agente.mu[0][-1] + agente.mu[0][0]*edad + agente.mu[0][1]*ingresos_generados

In [ ]:
# Valoración Q(s,a), según el modelo lineal, de realizar la
# acción 1 (promoción 1) en ese cliente concreto:
agente.mu[1][-1] + agente.mu[1][0]*edad + agente.mu[1][1]*ingresos_generados

In [ ]:
# Valoración Q(s,a), según el modelo lineal, de realizar la
# acción 2 (promoción 2) en ese cliente concreto:
agente.mu[2][-1] + agente.mu[2][0]*edad + agente.mu[2][1]*ingresos_generados